In [17]:
import sys
import copy
import math
import random
rook_count = 3

class Board:
    def __init__(self, board_size, goal):
        self.board_size = board_size
        self.goal = goal
        self.fitness = 0
        self.queens = []
        self.queens_index = {}
        for i in range(self.board_size-rook_count):
            s = "Q"+str(i)
            self.queens.append(s)
            self.queens_index[s] = i
            
        for i in range(rook_count):
            s = "R"+str(i)
            self.queens.append(s)
            self.queens_index[s] = i+self.board_size-rook_count
        '''
        for i in range(len(self.queens)):
            print(self.queens[i], "  pos = ", self.queens_index[self.queens[i]])
        
        self.queens=list(range(self.board_size-rook_count))
        self.rooks=list(range(rook_count))
        '''
        self.switch(self.board_size/2)

    def __del__(self):
        pass

    def switch(self, count):
        count=int(count)

        for i in range(count):
            j = random.randint(0, self.board_size-1)
            k = random.randint(0, self.board_size-1)
            #self.queens_index[self.queens[j]],self.queens_index[self.queens[k]] =  self.queens_index[self.queens[k]], self.queens_index[self.queens[j]]
            self.queens[j], self.queens[k] = self.queens[k], self.queens[j]

        self.compute_fitness()

    def regenerate(self):
        self.switch(2)

        if random.uniform(0,1) < 0.25:
            self.switch(1)

    def compute_fitness(self):
        self.fitness = self.goal
        for i in range(self.board_size):
            for j in range(i+1, self.board_size):
                #if(self.queens[i][0] == 'Q'):
                if math.fabs(self.queens_index[self.queens[i]] - self.queens_index[self.queens[j]]) == j - i:
                    self.fitness-=1
                '''
                elif(self.queens[i][0] == 'R'):
                    if(self.queens_index[self.queens[i]] == self.queens_index[self.queens[j]]):
                        self.fitness-=1
                '''
    def print_board(self):
        ''' prints current board in a nice way!'''
        for row in range(self.board_size):
            print ("", end="|")
            s = self.queens[row]
            idx = self.queens_index[s]
            for col in range(self.board_size):
                if col == idx:
                    if(s[0] == 'Q'):
                        print ("Q", end="|")
                    else:
                        print ("R", end="|")
                else:
                    print ("_", end="|")
            print ("")

class GaQueens:
    def __init__(self, board_size, population_size, generation_size):
        self.board_size = board_size
        self.population_size = population_size
        self.generation_size = generation_size

        self.generation_count = 0
        self.goal = int((self.board_size*(self.board_size-1))/2)
        self.population = []
        self.first_generation()

        while True:
            if self.is_goal_reached() == True:
                break
            if -1 < self.generation_size <= self.generation_count:
                break
            self.next_generation()

        print ("==================================================================")

        if -1 < self.generation_size <= self.generation_count:
            print ("Couldn't find result in %d generations" % self.generation_count)
        elif self.is_goal_reached():
            print ("Correct Answer found in generation %s" % self.generation_count)
            for population in self.population:
                if population.fitness == self.goal:
                    print (population.queens)
                    population.print_board()

    def __del__(self):
        pass

    def is_goal_reached(self):
        for population in self.population:
            if population.fitness == self.goal:
                return True
        return False

    def random_selection(self):
        population_list = []
        for  i in range(len(self.population)):
            population_list.append((i, self.population[i].fitness))
        population_list.sort(key=lambda pop_item: pop_item[1], reverse=True)
        return population_list[:int(len(population_list)/3)]

    def first_generation(self):
        ''' creates the first generation '''
        for i in range(self.population_size):
            self.population.append(Board(self.board_size, self.goal))

        self.print_cross()

    def next_generation(self):
        ''' creates next generations (all except first one)'''

        # add to generation counter
        self.generation_count+=1

        # get a list of selections to create next generation
        selections = self.random_selection()

        # creates a new population using given selection
        new_population = []
        while len(new_population) < self.population_size:
            sel = random.choice(selections)[0]
            new_population.append(copy.deepcopy(self.population[sel]))
        self.population = new_population

        for population in self.population:
            population.regenerate()

        self.print_cross(selections)

    def print_cross(self, selections=None):
        print ("Cross over #%d" % self.generation_count)

        if selections == None:
            selections = []

        print ("       Using: %s" % str([sel[0] for sel in selections]))

        count = 0
        for population in self.population:
            print ("%8d : (%d) %s" % (count, population.fitness, str(population.queens)))
            population.print_board()
            count+=1
            
if __name__ == '__main__':
    board_size = 8
    population_size = 20
    generation_size = -1

    # if there is arguments use them instead of default values
    if len(sys.argv) == 4:
        board_size = int(sys.argv[1])
        population_size = int(sys.argv[2])
        generation_size = int(sys.argv[3])

    # print some information about current quest!
    #print ("Starting:")
    #print ("    board size      : ", board_size)
    #print ("    population size : ", population_size)
    #print ("    generation size : ", generation_size)
    #print ("==================================================================")

    GaQueens(board_size, population_size, generation_size)


Cross over #0
       Using: []
       0 : (10) ['Q4', 'Q1', 'Q2', 'Q3', 'Q0', 'R0', 'R1', 'R2']
|_|_|_|_|Q|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|_|_|R|
       1 : (22) ['Q4', 'R0', 'R2', 'Q1', 'Q0', 'Q3', 'R1', 'Q2']
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|_|R|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|Q|_|_|_|_|_|
       2 : (20) ['Q2', 'Q1', 'Q0', 'R1', 'R0', 'Q3', 'Q4', 'R2']
|_|_|Q|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|_|R|
       3 : (26) ['R0', 'Q3', 'Q1', 'Q4', 'Q2', 'Q0', 'R1', 'R2']
|_|_|_|_|_|R|_|_|
|_|_|_|Q|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|Q|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|_|_|R|
       4 : (24) ['R0', 'R1', 'Q2', 'Q1', 'Q4', 'R2', 'Q3', 'Q0']
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|R|_|
|_|_|Q|_|_|_|_|_|
|_|Q|_|_|_|_|_

|_|_|_|_|_|_|R|_|
|Q|_|_|_|_|_|_|_|
|_|_|Q|_|_|_|_|_|
       5 : (22) ['Q4', 'Q2', 'Q3', 'R2', 'R1', 'R0', 'Q1', 'Q0']
|_|_|_|_|Q|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
       6 : (22) ['R0', 'Q2', 'Q3', 'R1', 'Q4', 'Q0', 'Q1', 'R2']
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|Q|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
       7 : (25) ['Q4', 'R0', 'Q3', 'Q1', 'R2', 'Q2', 'R1', 'Q0']
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|Q|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|Q|_|_|_|_|_|_|_|
       8 : (21) ['Q0', 'R0', 'Q2', 'Q3', 'R2', 'Q4', 'R1', 'Q1']
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
       9 : (22) ['Q3', 'R0', 'Q1', 'R2', 'Q2', 'Q0', 'Q4', 'R1']
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|Q|_|_|

|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|_|R|
      19 : (24) ['Q3', 'R0', 'Q0', 'Q2', 'R2', 'Q1', 'Q4', 'R1']
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|R|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|R|_|
Cross over #7
       Using: [0, 9, 8, 16, 18, 4]
       0 : (23) ['Q0', 'R1', 'R0', 'Q2', 'Q3', 'Q1', 'Q4', 'R2']
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|_|R|
       1 : (25) ['Q0', 'Q4', 'R0', 'Q2', 'R1', 'Q1', 'Q3', 'R2']
|Q|_|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
       2 : (22) ['Q4', 'R0', 'Q1', 'R2', 'R1', 'Q2', 'Q3', 'Q0']
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|R|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|_|_|R|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|Q|_|_|_|_|_|_|_|
       3 : (23) ['R0', 'Q3', 'Q1', 'Q2', 'Q4', 'R2', 'Q0', 'R

|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
      13 : (22) ['R0', 'Q3', 'Q4', 'Q2', 'R2', 'Q1', 'Q0', 'R1']
|_|_|_|_|_|R|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
      14 : (21) ['Q4', 'Q3', 'R1', 'R0', 'Q2', 'Q1', 'Q0', 'R2']
|_|_|_|_|Q|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
      15 : (26) ['Q4', 'R0', 'Q1', 'Q2', 'R1', 'Q3', 'R2', 'Q0']
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|R|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|Q|_|_|_|_|_|_|_|
      16 : (25) ['Q4', 'Q1', 'R0', 'R1', 'Q2', 'Q3', 'R2', 'Q0']
|_|_|_|_|Q|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|R|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|Q|_|_|_|_|_|_|_|
      17 : (24) ['Q4', 'R0', 'Q0', 'Q2', 'Q1', 'R2', 'Q3', 'R1']
|_|_|_|_|Q|_|_|_|
|_|_|_|_|

|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|_|R|
|_|Q|_|_|_|_|_|_|
      13 : (24) ['Q2', 'Q1', 'R0', 'R1', 'Q3', 'R2', 'Q0', 'Q4']
|_|_|Q|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
      14 : (23) ['Q2', 'Q0', 'R1', 'R2', 'Q3', 'Q1', 'R0', 'Q4']
|_|_|Q|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|_|_|R|
|_|_|_|Q|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|Q|_|_|_|
      15 : (26) ['Q4', 'R2', 'Q3', 'R1', 'Q1', 'R0', 'Q0', 'Q2']
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|Q|_|_|_|_|_|
      16 : (24) ['Q3', 'R0', 'R2', 'R1', 'Q2', 'Q0', 'Q1', 'Q4']
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|_|_|R|_|
|_|_|Q|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
      17 : (24) ['R2', 'Q1', 'R0', 'R1', 'Q2', 'Q4', 'Q0', 'Q3']
|_|_|_|_|_|_|_|R|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|

|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
      13 : (27) ['R0', 'Q0', 'R1', 'Q4', 'Q2', 'Q1', 'Q3', 'R2']
|_|_|_|_|_|R|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|Q|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
      14 : (26) ['R1', 'Q2', 'Q0', 'R0', 'R2', 'Q4', 'Q3', 'Q1']
|_|_|_|_|_|_|R|_|
|_|_|Q|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|Q|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|Q|_|_|_|_|_|_|
      15 : (22) ['Q0', 'R2', 'Q1', 'Q4', 'Q3', 'Q2', 'R1', 'R0']
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
      16 : (23) ['Q0', 'R1', 'R0', 'Q4', 'Q2', 'Q1', 'Q3', 'R2']
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|_|_|R|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|Q|_|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
      17 : (23) ['Q3', 'Q1', 'Q0', 'R2', 'Q2', 'R1', 'R0', 'Q4']
|_|_|_|Q|_|_|_|_|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|

      13 : (22) ['R1', 'Q1', 'Q0', 'Q3', 'R2', 'R0', 'Q2', 'Q4']
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
      14 : (25) ['R0', 'Q2', 'Q0', 'R1', 'Q1', 'Q4', 'R2', 'Q3']
|_|_|_|_|_|R|_|_|
|_|_|Q|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|Q|_|_|_|_|
      15 : (26) ['R1', 'Q1', 'Q3', 'Q0', 'R2', 'Q2', 'Q4', 'R0']
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|Q|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|R|_|_|
      16 : (24) ['Q2', 'Q4', 'R2', 'R1', 'Q1', 'R0', 'Q0', 'Q3']
|_|_|Q|_|_|_|_|_|
|_|_|_|_|Q|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|_|_|R|_|
|_|Q|_|_|_|_|_|_|
|_|_|_|_|_|R|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|Q|_|_|_|_|
      17 : (24) ['Q1', 'Q0', 'R1', 'Q3', 'R2', 'R0', 'Q2', 'Q4']
|_|Q|_|_|_|_|_|_|
|Q|_|_|_|_|_|_|_|
|_|_|_|_|_|_|R|_|
|_|_|_|Q|_|_|_|_|
|_|_|_|_|_|_|_|R|
|_|_|_|_|